<h1 align="center"><strong>FEniCS $\rightarrow$ Ecuación de Poisson</strong></h1>

<div align="justify">
    La ecuaciòn de Poisson es un <i>problema de valor frontera</i> empleado para la solución de problemas de:
</div>

* Conducción de calor.
* Electrostática.
* Difusión de sustancias.
* Flujo de fluidos.
* Navier-Stokes.

<div align="justify">
    Y se puede apreciar en la Ecuación \ref{Poisson}.
    $$
    \begin{equation}
        -\nabla ^2 u(x) = f(x) \rightarrow x \text{ en } \Omega \\
        u(x) = u_D (x) \rightarrow x \text{ sobre } d\Omega
        \tag{1}
        \label{Poisson}
    \end{equation}
    $$
    Dónde: $u(x)$ es una función desconocida, llamada también <i>función de juicio</i> (trial); $f(x)$ es una función preescrita; $\Omega$ es el dominio de estudio; $u_D$ se refiere a la condición de frontera (tipo <i>Dirichlet</i>) sobre el contorno $d\Omega$.
</div>

<br>

<div align="justify">
    El punto de partida para solucionar el problema mediante elementos finitos, consiste en expresar la Ecuación \ref{Poisson} en la <i>forma variacional</i>, como se muestra en la Ecuación \ref{var}.
    $$
    \begin{equation}
        -\int _{\Omega} \left(\nabla ^2 u \right) v \, dx = \int _{\Omega} f v \, dx
        \tag{2}
        \label{var}
    \end{equation}
    $$
    Dónde: $v(x)$ es una <i>función de prueba</i>.
    <br>
    <br>
    La solución de la ecuación \ref{var}, mediante elementos finitos, se realiza a través de la <strong>notación canónica</strong> de problemas variacionales expresados en la <i>forma débil</i>; de modo que:
    $$
    \begin{equation}
        a\left(u,v \right) = L(v) \, \, \, \, \, \, \forall v \in \hat{V}
        \tag{2}
    \end{equation}
    $$
    Dónde:
    $$
    \begin{equation}
        a\left(u,v \right) = \int _{\Omega} \nabla u \cdot \nabla v \, dx
        \tag{3}
    \end{equation}
    $$
    <br>
    $$
    \begin{equation}
        L(v) = \int _{\Omega} f \, v \, dx
        \tag{4}
    \end{equation}
    $$
    Para reproducir la solución exacta, se deben tener datos referentes al dominio del problema $\left(\Omega \right)$, condiciones de frontera $\left(u_D \right)$ y el término fuente $(f)$. 
</div>

## __Ejemplo__

$$
\begin{equation}
u_e (x,y) = 1+x^2+2y^2
\tag{5}
\label{ejemplo}
\end{equation}
$$

<div align="justify">
    Al insertar la Ecuación \ref{ejemplo} en la Ecuación \ref{Poisson}, nos encontramos con que $u_e$ es una solución si:
    $$
    \begin{equation}
        f(x,y) = -6, \, \, \, \, u_D(x,y) = u_e(x,y) = 1+x^2+2y^2 \\
        \Omega = [0,1] x [0,1]
    \end{equation}
    $$
    El error de la simulación es calculado a través de la siguiente relación matemática:
    $$
    \begin{equation}
        E = \sqrt{\int _{\Omega} \left(u_D - u \right)^2 dx}
    \end{equation}
    $$
</div>

In [ ]:
%matplotlib widget
from fenics import *
import numpy as np

#Creación de malla y definición del espacio
mesh = UnitSquareMesh(8,8)    #8x8 se refiere a las subdivisiones del dominio.
V = FunctionSpace(mesh, 'P', 1)

#Condiciones de frontera
u_D = Expression('1 + x[0]*x[0] + 2*x[1]*x[1]', degree=2)

def boundary(x, on_boundary):
    #Retorna True si el punto x está en la condición de frontera.
    return on_boundary

bc = DirichletBC(V, u_D, boundary) 

#Definición del problema variacional
u = TrialFunction(V)
v = TestFunction(V)
f = Constant(-6.0)          #Término fuente. Es equivalente: f = Expression('-6', degree=0)
a = dot(grad(u), grad(v))*dx  #Especifica la PDE a resolver  
L = f*v*dx

#Computación de soluciones
u = Function(V)    #Se redefine la variable u (reemplaza su significado)
solve(a == L, u, bc)

#Graficar solución y malla
plot(u)
plot(mesh)

#Guardar solución en un archivo formato VTK
vtkfile = File('poisson/solution.pvd')
vtkfile << u

#Computación de error en normativa L2
error_L2 = errornorm(u_D, u, 'L2')

#Computación máxima del error en los vértices
vertex_values_u_D = u_D.compute_vertex_values(mesh)
vertex_values_u = u.compute_vertex_values(mesh)
error_max = np.max(np.abs(vertex_values_u_D - vertex_values_u))

#Impresión de errores
print('error_L2 = ', error_L2)
print('error_max = ', error_max)

#Mantener la gráfica
#interactive()

Para visualización de objetos renderizados multidimensionales están:
* _vtk.js_ con _itkwidgets_.

```python
from itkwidgets import view
import itk
file = 'poisson/solution.pvd'
image = itk.imread(file)
#view('poisson/solution.pvd')
```


* pyvista
* PVGeo
* ParaView

In [ ]:
%matplotlib inline
from pyvista import set_plot_theme
import pyvista as pv
import numpy as np
import matplotlib.pyplot as plt
set_plot_theme('document')

filename = 'poisson/solution.pvd'
mesh = pv.read('Seeder.iges')
cpos = mesh.plot()

In [ ]:
import pyvista
from pyvista import examples
from PVGeo.model_build import CreateTensorMesh
from PVGeo.grids import ExtractTopography
import os

###############################################################################
# For the grid data set, let's use one of the Model Building sources
# with the following parameters:
#
# - Origin: ``[793000, 9192500, 2690]``
# - X Cells: ``'1000 500 50*250 500 1000'``
# - Y Cells: ``'1000 500 55*250 500 1000'``
# - Z Cells: ``'30*100.0 5*250.0 500'``

mesh = CreateTensorMesh(origin=[793000, 9192500, 2690],
                        xcellstr='1000 500 50*250 500 1000',
                        ycellstr='1000 500 55*250 500 1000',
                        zcellstr='30*100.0 5*250.0 500').apply()

mesh.plot(show_grid=True, color=True, show_edges=True)

###############################################################################
# Now load the topography file from the example data:
link = 'https://dl.dropbox.com/s/gw5v3tiq68oge3l/Example-Extract-Topo.zip?dl=0'
examples.downloads._retrieve_file(link, 'Example-Extract-Topo.zip')
topo = pyvista.read(os.path.join(pyvista.EXAMPLES_PATH, 'topo.vtk'))

p = pyvista.Plotter()
p.add_mesh(topo, cmap='terrain')
p.add_mesh(mesh, color=True, show_edges=False, opacity=0.75)
p.show_grid()
p.show()

###############################################################################
# Now that you have the topography and a grid data set,
# let's go ahead and use the **Extract Topography** filter. Be sure to properly
# select the inputs to the algorithm.
extracted = ExtractTopography().apply(mesh, topo)
extracted.plot(scalars='Extracted')

###############################################################################
# op='underneath', tolerance=0.001, offset=0.0, invert=False, remove=False
# This will show the cells that are active underneath the topography surface
# (0 for above surface and 1 for below surface). Now we can threshold this gridded
# data set to remove parts of the model that are above the topography surface by
# applying a *Threshold* filter to chop out all values below 1.
#
# The resulting grid with cells above the topography extracted will look like the
# rendering below:
threshed = extracted.threshold(0.5)
threshed.plot(color=True, show_edges=True)

###############################################################################
# How well did this remove cells above the topography surface?

p = pyvista.Plotter()
p.add_mesh(topo, cmap='terrain')
p.add_mesh(threshed, color=True, show_edges=True)
p.show_grid()
p.show()

###############################################################################
# Is that extraction too close to the topography surface? To better extract the
# topographic surface, you can set a tolerance:
extracted = ExtractTopography(tolerance=100., remove=True).apply(mesh, topo)

p = pyvista.Plotter()
p.add_mesh(topo, cmap='terrain')
p.add_mesh(extracted, color=True, show_edges=True)
p.show_grid()
p.show()

###############################################################################
# Note that there are other extraction operations like an ``'intersection'``:
extracted = ExtractTopography(op='intersection',
                              remove=True,
                              tolerance=100.).apply(mesh, topo)
extracted.plot(color=True, show_edges=True)